In [97]:
import pandas as pd
import statsmodels.api as sm

spy_min = pd.read_parquet("SPY_combined_minute_data.parquet")  
nge_daily = pd.read_csv("daily_nge.csv")

In [99]:
spy_min['timestamp'] = spy_min['timestamp'].dt.tz_convert('US/Eastern')
spy_min["date"] = spy_min["timestamp"].dt.date
spy_min["hour"] = spy_min["timestamp"].dt.hour
spy_min["minute"] = spy_min["timestamp"].dt.minute
spy_min

,timestamp,open,high,low,close,date,year,month,day,hour,minute
0,2018-08-31 09:30:00-04:00,289.869995,289.885010,289.790009,289.850006,2018-08-31,2018,8,31,9,30
1,2018-08-31 09:31:00-04:00,289.834991,289.970001,289.820007,289.950012,2018-08-31,2018,8,31,9,31
2,2018-08-31 09:32:00-04:00,289.940002,290.040009,289.920013,290.040009,2018-08-31,2018,8,31,9,32
3,2018-08-31 09:33:00-04:00,290.029999,290.054993,289.950012,289.989990,2018-08-31,2018,8,31,9,33
4,2018-08-31 09:34:00-04:00,289.964996,290.035004,289.950012,289.975006,2018-08-31,2018,8,31,9,34
...,...,...,...,...,...,...,...,...,...,...,...
506036,2023-08-31 15:55:00-04:00,450.915009,450.924988,450.600006,450.834991,2023-08-31,2023,8,31,15,55
506037,2023-08-31 15:56:00-04:00,450.820007,450.820007,450.649994,450.709991,2023-08-31,2023,8,31,15,56
506038,2023-08-31 15:57:00-04:00,450.709991,450.744995,450.595001,450.670013,2023-08-31,2023,8,31,15,57
506039,2023-08-31 15:58:00-04:00,450.670013,450.804993,450.584991,450.665009,2023-08-31,2023,8,31,15,58


In [107]:
prev_close = spy_min[(spy_min["hour"] == 15) & (spy_min["minute"] == 59)][["date", "close"]].rename(columns={"close": "prev_close"})
prev_close['prev_close'] = prev_close['prev_close'].shift(1)
prev_close = prev_close.dropna(subset=["prev_close"])
close_1430 = spy_min[(spy_min["hour"] == 14) & (spy_min["minute"] == 30)][["date", "close"]].rename(columns={"close": "close_1430"})
close_1530 = spy_min[(spy_min["hour"] == 15) & (spy_min["minute"] == 30)][["date", "close"]].rename(columns={"close": "close_1530"})
close_1559 = spy_min[(spy_min["hour"] == 15) & (spy_min["minute"] == 59)][["date", "close"]].rename(columns={"close": "close_1559"})
intraday = prev_close.merge(close_1430, on="date").merge(close_1530, on="date").merge(close_1559, on="date")
intraday["rROD"] = intraday["close_1430"] / intraday["prev_close"] - 1
intraday["rLH"] = intraday["close_1530"] / intraday["close_1559"] - 1
nge_daily["date"] = pd.to_datetime(nge_daily["date"]).dt.date
data = intraday.merge(nge_daily, on="date", how="inner")
data

,date,prev_close,close_1430,close_1530,close_1559,rROD,rLH,NGE_raw,Pt,MV_t,NGE
0,2018-09-04,290.220001,289.720001,289.725006,289.855011,-0.001723,-0.000449,1.395750e+08,259.434570,2.464628e+11,0.000566
1,2018-09-05,289.855011,289.059998,289.059998,289.049988,-0.002743,0.000035,1.080051e+08,258.736298,2.457995e+11,0.000439
2,2018-09-06,289.049988,287.894989,288.484985,288.179993,-0.003996,0.001058,7.010234e+07,257.957489,2.450596e+11,0.000286
3,2018-09-07,288.179993,287.795013,287.815002,287.700012,-0.001336,0.000400,2.563834e+07,257.456207,2.445834e+11,0.000105
4,2018-09-10,287.700012,288.345001,288.260010,288.100006,0.002242,0.000555,4.936117e+07,257.903748,2.450086e+11,0.000201
...,...,...,...,...,...,...,...,...,...,...,...
1251,2023-08-25,436.894989,440.325012,440.894989,440.059998,0.007851,0.001897,-5.343755e+07,428.563019,4.071349e+11,-0.000131
1252,2023-08-28,440.059998,441.299988,441.674988,442.700012,0.002818,-0.002315,-2.095173e+07,431.280731,4.097167e+11,-0.000051
1253,2023-08-29,442.700012,448.380005,449.170013,449.165009,0.012830,0.000011,3.688119e+07,437.514771,4.156390e+11,0.000089
1254,2023-08-30,449.165009,450.220001,450.980011,451.010010,0.002349,-0.000067,5.135533e+07,439.316833,4.173510e+11,0.000123


In [109]:
import statsmodels.formula.api as smf
data["interaction"] = data["rROD"] * data["NGE"]
model = smf.ols("rLH ~ rROD + NGE + interaction", data=data).fit()
def run_regression(df, x_var, y_var):
    X = sm.add_constant(df[x_var])
    y = df[y_var]
    model = sm.OLS(y, X).fit()
    return model.summary()

pos_nge = data[data["NGE"] > 0]
neg_nge = data[data["NGE"] < 0]

print("📘 Regression when NGE > 0")
print(run_regression(pos_nge, "rROD", "rLH"))

print("\n📙 Regression when NGE < 0")
print(run_regression(neg_nge, "rROD", "rLH"))

print(model.summary())

📘 Regression when NGE > 0
                            OLS Regression Results                            
Dep. Variable:                    rLH   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     9.994
Date:                Tue, 05 Aug 2025   Prob (F-statistic):            0.00164
Time:                        19:08:14   Log-Likelihood:                 3516.7
No. Observations:                 734   AIC:                            -7029.
Df Residuals:                     732   BIC:                            -7020.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0001   7.

In [111]:
data.to_csv('nge_intraday_data.csv')

In [113]:
data

,date,prev_close,close_1430,close_1530,close_1559,rROD,rLH,NGE_raw,Pt,MV_t,NGE,interaction
0,2018-09-04,290.220001,289.720001,289.725006,289.855011,-0.001723,-0.000449,1.395750e+08,259.434570,2.464628e+11,0.000566,-9.756503e-07
1,2018-09-05,289.855011,289.059998,289.059998,289.049988,-0.002743,0.000035,1.080051e+08,258.736298,2.457995e+11,0.000439,-1.205181e-06
2,2018-09-06,289.049988,287.894989,288.484985,288.179993,-0.003996,0.001058,7.010234e+07,257.957489,2.450596e+11,0.000286,-1.143058e-06
3,2018-09-07,288.179993,287.795013,287.815002,287.700012,-0.001336,0.000400,2.563834e+07,257.456207,2.445834e+11,0.000105,-1.400371e-07
4,2018-09-10,287.700012,288.345001,288.260010,288.100006,0.002242,0.000555,4.936117e+07,257.903748,2.450086e+11,0.000201,4.516590e-07
...,...,...,...,...,...,...,...,...,...,...,...,...
1251,2023-08-25,436.894989,440.325012,440.894989,440.059998,0.007851,0.001897,-5.343755e+07,428.563019,4.071349e+11,-0.000131,-1.030450e-06
1252,2023-08-28,440.059998,441.299988,441.674988,442.700012,0.002818,-0.002315,-2.095173e+07,431.280731,4.097167e+11,-0.000051,-1.440916e-07
1253,2023-08-29,442.700012,448.380005,449.170013,449.165009,0.012830,0.000011,3.688119e+07,437.514771,4.156390e+11,0.000089,1.138487e-06
1254,2023-08-30,449.165009,450.220001,450.980011,451.010010,0.002349,-0.000067,5.135533e+07,439.316833,4.173510e+11,0.000123,2.890191e-07
